<a href="https://colab.research.google.com/github/peremartra/fairness-pruning/blob/main/notebooks/02_Evaluate_1B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fairness Pruning Research - Base Model Evaluation
## 02 - Comprehensive Benchmark Suite for Unpruned Models

### Establishing Performance Baselines for Bias Mitigation Research
by [Pere Martra](https://github.com/peremartra)

[![GitHub](https://img.shields.io/badge/⭐_Star-OptiPFair-orange?logo=github&logoColor=white)](https://github.com/peremartra/optipfair)
[![PyPI](https://img.shields.io/pypi/v/optipfair?logo=python&logoColor=white&label=v)](https://pypi.org/project/optipfair/)

**Repository:** [github.com/peremartra/fairness-pruning](https://github.com/peremartra/fairness-pruning)

---

**Colab Environment:** GPU L4 or A100

**Models to Evaluate:**
* Llama-3.2-1B (base)
* Llama-3.2-3B (base)
* Additional models defined in `EXPERIMENT_CONFIG`

**Benchmarks (15 total):**
* English: MMLU, HellaSwag, BoolQ, ARC-Challenge, WinoGrande, PIQA, TruthfulQA, GSM8K, IFEval, MUSR
* Spanish: Belebele, XCOPA, MMLU-ES
* Language Modeling: WikiText, Lambada-OpenAI

**Estimated Runtime:** ~3-4 hours (varies by number of models)

---

## 📋 Objective

Establish **performance baselines** for the Fairness Pruning project by evaluating unpruned base models.

**Purpose:**
1. Measure baseline performance before bias mitigation interventions
2. Create reference metrics for future pruned model comparisons
3. Validate benchmark configurations across different architectures
4. Capture cross-lingual performance (English + Spanish)

**Features:**
- ✅ Checkpoint/Resume Support (survives Colab disconnections)
- ✅ Multi-Model Support (generic, not 1B-specific)
- ✅ Robust Error Handling (continues on task failures)
- ✅ Automated Path Management (no manual configuration needed)

**Note:** This notebook evaluates ONLY base models (no pruning applied). For bias mitigation experiments with pruned models, see subsequent notebooks.

---

# 1. Setup & Installation

In [1]:
# Install required libraries
!pip install -q optipfair
!pip install -q lm-eval
!pip install -q langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 50.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# Mount Google Drive for checkpoint persistence
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download utils.py from GitHub repository
!wget -q https://raw.githubusercontent.com/peremartra/fairness-pruning/main/utils.py

# Verify download
import os
if os.path.exists('utils.py'):
    print("✅ utils.py downloaded successfully")
else:
    print("❌ Failed to download utils.py")

In [4]:
# Import core libraries and utilities
import torch
import json
import pandas as pd
from datetime import datetime
from pathlib import Path

# Import our utility functions
from utils import (
    EXPERIMENT_CONFIG,
    BENCHMARKS_BASE,
    load_or_create_model,
    run_robust_evaluation,
    clear_gpu_cache,
    get_model_stats,
    format_results_table
)

print("✅ All imports successful")
print(f"📱 Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

✅ All imports successful
📱 Device: GPU
   GPU: NVIDIA L4
   Memory: 23.8 GB


# 1. Helper Functions

Utility functions for automatic checkpoint path generation and model size detection.

In [ ]:
import re
import os

def get_model_size(model_name: str) -> str:
    """Extract model size identifier from HuggingFace model name.
    
    Examples:
        "meta-llama/Llama-3.2-1B" → "1b"
        "meta-llama/Llama-3.2-3B-Instruct" → "3b_instruct"
        "BSC-LT/salamandra-2b" → "2b"
    """
    match = re.search(r'(\d+\.?\d*)[Bb]', model_name)
    if not match:
        return "unknown"
    
    size = match.group(1).replace('.', '_') + "b"
    if "instruct" in model_name.lower():
        size += "_instruct"
    
    return size.lower()

def get_checkpoint_path(model_name: str, base_dir: str) -> str:
    """Generate checkpoint path with size-based subdirectory.
    
    Args:
        model_name: Full HuggingFace model identifier
        base_dir: Base directory for checkpoints
        
    Returns:
        Full path to checkpoint file
    """
    model_size = get_model_size(model_name)
    safe_name = model_name.replace('/', '_').replace('-', '_').lower()
    checkpoint_dir = os.path.join(base_dir, model_size)
    os.makedirs(checkpoint_dir, exist_ok=True)
    return os.path.join(checkpoint_dir, f"{safe_name}.json")

# Test with EXPERIMENT_CONFIG
print("Testing helper functions with EXPERIMENT_CONFIG:")
print("-" * 70)
for cfg in EXPERIMENT_CONFIG:
    model_id = cfg['base_model']
    size = get_model_size(model_id)
    print(f"{model_id:<50} → {size}")
print("-" * 70)

# 2. Configuration & Evaluation Plan

This section prepares the evaluation for all models defined in `EXPERIMENT_CONFIG`.

In [ ]:
# Directory setup
CHECKPOINT_BASE_DIR = "/content/drive/MyDrive/glu_pruning/checkpoints"
RESULTS_DIR = "/content/drive/MyDrive/glu_pruning/results"
Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)

# De-duplicate models from EXPERIMENT_CONFIG
unique_models = list(dict.fromkeys([cfg["base_model"] for cfg in EXPERIMENT_CONFIG]))

print(f"{'='*70}")
print("📊 EVALUATION PLAN: Base Model Benchmarking")
print(f"{'='*70}\n")
print(f"Models to evaluate: {len(unique_models)}")
print(f"Benchmarks per model: {len(BENCHMARKS_BASE)}")
print(f"Total evaluations: {len(unique_models) * len(BENCHMARKS_BASE)}")
print(f"Estimated time: ~{len(unique_models) * 1.5:.1f} hours\n")

# Display models with checkpoint status
print("Models to evaluate:")
print("-" * 70)
print(f"{'Model ID':<50} {'Size':<10} {'Status'}")
print("-" * 70)
for model_id in unique_models:
    size = get_model_size(model_id)
    cp_path = get_checkpoint_path(model_id, CHECKPOINT_BASE_DIR)
    exists = "✅ Exists" if Path(cp_path).exists() else "🆕 New"
    print(f"{model_id:<50} {size:<10} {exists}")
print("-" * 70)

# Display benchmarks
print("\nBenchmarks:")
print("-" * 70)
for i, task in enumerate(BENCHMARKS_BASE, 1):
    fewshot_str = f"{task['num_fewshot']}-shot"
    print(f"{i:2d}. {task['name']:<30} {fewshot_str}")
print("-" * 70)

print(f"\n⚙️  Configuration:")
print(f"   - Checkpointing: Enabled (per-task granularity)")
print(f"   - Auto-resume: Yes (survives disconnections)")
print(f"   - Error handling: Skip failed tasks, continue evaluation")
print(f"   - Device: {'GPU' if torch.cuda.is_available() else 'CPU'}\n")

# 3. Base Model Evaluation

Evaluates each base model across all benchmarks with checkpoint/resume support.

**Process:**
1. Load model directly from HuggingFace Hub (no pruning applied)
2. Calculate model statistics (parameters, size)
3. Run evaluation with checkpoint system (saves progress after each task)
4. Clear GPU memory before next model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

print(f"\n{'='*70}")
print("🚀 STARTING EVALUATION")
print(f"{'='*70}\n")

all_model_results = {}

for i, model_id in enumerate(unique_models, 1):
    print(f"\n{'='*70}")
    print(f"📊 MODEL {i}/{len(unique_models)}: {model_id}")
    print(f"{'='*70}\n")
    
    try:
        # 1. Load model from HuggingFace Hub (NO pruning)
        print(f"Loading from HuggingFace Hub...")
        model = AutoModelForCausalLM.from_pretrained(
            model_id, 
            torch_dtype=torch.bfloat16,  # Use bfloat16 for A100, float16 for T4/L4
            device_map="auto"
        )
        
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        print("✅ Model loaded successfully\n")
        
        # 2. Display model statistics
        stats = get_model_stats(model)
        print(f"📈 Model Statistics:")
        print(f"   Parameters: {stats['total_parameters']:,}")
        print(f"   Size: {stats['size_gb']:.2f} GB\n")
        
        # 3. Generate checkpoint path automatically
        checkpoint_path = get_checkpoint_path(model_id, CHECKPOINT_BASE_DIR)
        print(f"📁 Checkpoint: {checkpoint_path}\n")
        
        # 4. Run evaluation with checkpoint/resume support
        results = run_robust_evaluation(
            model=model,
            tokenizer=tokenizer,
            tasks=BENCHMARKS_BASE,
            checkpoint_path=checkpoint_path,
            model_name=model_id
        )
        
        all_model_results[model_id] = results
        
        print(f"\n✅ Completed: {model_id}")
        print("\nResults Preview:")
        print(format_results_table(results))
        
        # 5. Cleanup memory before next model
        del model, tokenizer
        clear_gpu_cache()
        
    except Exception as e:
        print(f"\n❌ ERROR evaluating {model_id}: {str(e)}")
        
        # Check for common issues
        if "401" in str(e) or "403" in str(e):
            print("   → Authentication required. Run: huggingface-cli login")
        elif "CUDA out of memory" in str(e):
            print("   → GPU OOM. Try reducing batch size or using smaller model")
        
        print("   → Continuing with next model...\n")
        
        # Cleanup and continue
        if 'model' in locals():
            del model
        if 'tokenizer' in locals():
            del tokenizer
        clear_gpu_cache()
        continue

print(f"\n{'='*70}")
print(f"✅ EVALUATION COMPLETE: {len(all_model_results)}/{len(unique_models)} models")
print(f"{'='*70}\n")

# 4. Results Consolidation

Load checkpoint files and consolidate into a single DataFrame for analysis.

In [ ]:
import glob

print(f"{'='*70}")
print("📊 CONSOLIDATING RESULTS")
print(f"{'='*70}\n")

# Find all checkpoint files recursively
checkpoint_files = glob.glob(f"{CHECKPOINT_BASE_DIR}/**/*.json", recursive=True)
print(f"Found {len(checkpoint_files)} checkpoint files\n")

consolidated_data = []

for json_path in sorted(checkpoint_files):
    print(f"  → Processing: {os.path.basename(json_path)}")
    
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
        
        # Extract metadata
        metadata = data.get("metadata", {})
        model_name = metadata.get("model_name", "Unknown")
        model_size = get_model_size(model_name)
        
        # Extract results for each task
        results = data.get("results", {})
        if not results:
            print(f"    ⚠️ No results found, skipping")
            continue
        
        # Process each task
        for task_name, metrics in results.items():
            row = {
                "model": model_name,
                "model_size": model_size,
                "task": task_name
            }
            
            # Add all metrics
            for metric_name, value in metrics.items():
                try:
                    row[metric_name] = float(value)
                except (ValueError, TypeError):
                    row[metric_name] = value
            
            consolidated_data.append(row)
    
    except Exception as e:
        print(f"    ⚠️ Error processing file: {e}")
        continue

# Create DataFrame
df = pd.DataFrame(consolidated_data)

if not df.empty:
    df = df.sort_values(by=["model", "task"]).reset_index(drop=True)
    print(f"\n✅ Consolidated {len(df)} result rows")
    print(f"   Models: {df['model'].nunique()}")
    print(f"   Tasks: {df['task'].nunique()}")
    print(f"   Metrics per task: {len(df.columns) - 3}")  # Exclude model, model_size, task
    
    print("\nDataFrame Preview:")
    print(df.head(15).to_string())
else:
    print("\n⚠️ No data consolidated")

In [ ]:
if not df.empty:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save detailed results CSV
    csv_path = f"{RESULTS_DIR}/base_models_results_{timestamp}.csv"
    df.to_csv(csv_path, index=False)
    print(f"\n💾 Results saved:")
    print(f"   {csv_path}")
    
    # Save latest version
    latest_csv = f"{RESULTS_DIR}/base_models_results_latest.csv"
    df.to_csv(latest_csv, index=False)
    print(f"   {latest_csv}")
    
    # Save JSON format
    json_path = f"{RESULTS_DIR}/base_models_results_{timestamp}.json"
    df.to_json(json_path, orient='records', indent=2)
    print(f"   {json_path}")
    
    print(f"\n✅ All results exported successfully")

# 5. Summary Analysis

Generate summary statistics comparing models.

In [ ]:
if not df.empty:
    print(f"{'='*70}")
    print("📈 SUMMARY STATISTICS")
    print(f"{'='*70}\n")
    
    summary = []
    for model_name, model_df in df.groupby('model'):
        # Calculate aggregated metrics
        acc = model_df['accuracy'].dropna()
        ppl = model_df['perplexity'].dropna()
        
        # Get model metadata (use first row since all rows have same metadata)
        model_size = model_df['model_size'].iloc[0] if 'model_size' in model_df.columns else get_model_size(model_name)
        
        summary.append({
            "model": model_name,
            "model_size": model_size,
            "avg_accuracy": acc.mean() if len(acc) > 0 else None,
            "avg_perplexity": ppl.mean() if len(ppl) > 0 else None,
            "tasks_completed": len(model_df),
            "tasks_with_accuracy": len(acc),
            "tasks_with_perplexity": len(ppl)
        })
    
    summary_df = pd.DataFrame(summary)
    summary_df = summary_df.sort_values("model").reset_index(drop=True)
    
    print(summary_df.to_string(index=False, float_format="%.4f"))
    
    # Save summary
    summary_csv = f"{RESULTS_DIR}/base_models_summary_{timestamp}.csv"
    summary_df.to_csv(summary_csv, index=False)
    
    print(f"\n💾 Summary saved: {summary_csv}")
    print(f"\n{'='*70}")

# 6. Evaluation Complete

## Summary

Baseline performance metrics established for the Fairness Pruning project.

**Generated Files:**
- `base_models_results_latest.csv` - Full evaluation results
- `base_models_results_YYYYMMDD_HHMMSS.json` - Structured export
- `base_models_summary_YYYYMMDD_HHMMSS.csv` - Summary metrics
- Individual checkpoint JSONs per model (in subdirectories by size)

**Next Steps:**
1. Use these baselines as reference for bias mitigation experiments
2. Identify high-variance tasks that may be sensitive to interventions
3. Proceed to bias detection and pruning notebooks

---

**Powered by OptiPFair** - Activation-Guided MLP Width Pruning for Bias Mitigation

If this research helps your work:
- ⭐ Star [the repo](https://github.com/peremartra/optipfair)
- 📖 Read the [documentation](https://peremartra.github.io/optipfair/)
- 🐛 Report issues or suggest features

---

In [ ]:
print(f"{'='*70}")
print("📁 GENERATED FILES")
print(f"{'='*70}\n")

print("Results:")
if 'csv_path' in locals() and os.path.exists(csv_path):
    print(f"  ✅ {csv_path}")
if 'latest_csv' in locals() and os.path.exists(latest_csv):
    print(f"  ✅ {latest_csv}")
if 'json_path' in locals() and os.path.exists(json_path):
    print(f"  ✅ {json_path}")
if 'summary_csv' in locals() and os.path.exists(summary_csv):
    print(f"  ✅ {summary_csv}")

print("\nCheckpoints:")
if 'checkpoint_files' in locals():
    for f in sorted(checkpoint_files)[:10]:  # Show first 10
        print(f"  ✅ {f}")
    if len(checkpoint_files) > 10:
        print(f"  ... and {len(checkpoint_files) - 10} more")

print(f"\n{'='*70}")
print("✅ EVALUATION COMPLETE")
print(f"{'='*70}")